<a href="https://colab.research.google.com/github/balugaddam/Wheat-Leaf-Disease-Detection/blob/main/Wheat_Leaf_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================================================
# WHEAT LEAF DISEASE DETECTION - FINAL CODE
# (Assumes Kaggle CLI Authentication is complete)
# ========================================================================

# 📦 Install dependencies
!pip install opencv-python tensorflow imutils scikit-learn matplotlib pandas kagglehub

# 📁 Import libraries
import cv2, os, pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
import glob # Needed to find the downloaded zip file name
import kagglehub

# --- TENSORFLOW IMPORTS ---
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
# ---------------------------------

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns # For confusion matrix/better plots

# --- CONFIGURATION (CRITICAL) ---
# IMPORTANT: Kagglehub handles the extraction, this folder will be set dynamically
# EXTRACT_FOLDER will be set to the path returned by kagglehub.dataset_download
EXTRACT_FOLDER = ""
TEST_SPLIT = 0.25
EPOCHS = 50
BATCH_SIZE = 64
# ---------------------------------

# --- DOWNLOAD DATASET USING KAGGLEHUB ---
print(f"[SETUP] Downloading dataset lyxbash/lcdcd-2020-dataset using KaggleHub...")
try:
    # kagglehub.dataset_download returns the path to the extracted dataset
    dataset_path = kagglehub.dataset_download("lyxbash/lcdcd-2020-dataset")
    EXTRACT_FOLDER = dataset_path
    print(f"[SUCCESS] Dataset downloaded and extracted to: {EXTRACT_FOLDER}")
except Exception as e:
    print(f"[FATAL] Error during dataset download using KaggleHub: {e}")
    raise SystemExit

# --- 🧠 Otsu Segmentation Function (Preprocessing) ---
def preprocess_with_otsu(image_path):
    """Loads, segments using Otsu's thresholding, and resizes an image."""
    image = cv2.imread(image_path)
    if image is None:
        return None
    # Segmentation to isolate the leaf
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return cv2.resize(segmented, (224, 224))

# --- PART 1: DATA LOADING AND PREPARATION ---
print("\n--- PART 1: Data Loading and Preparation ---")

# Dynamically determine LABELS from the dataset structure
LABELS = set()
base_search_path = EXTRACT_FOLDER

# Check for single nested directory that might be the true root for classes
contents_in_root = [os.path.join(EXTRACT_FOLDER, d) for d in os.listdir(EXTRACT_FOLDER)]
directories_in_root = [d for d in contents_in_root if os.path.isdir(d)]

# If there's only one directory in the extracted folder, and it's not a typical 'train'/'test' split,
# assume it's an extra wrapping folder and use it as the new base search path.
if len(directories_in_root) == 1:
    single_subdir_name = os.path.basename(directories_in_root[0])
    # Check if this single subdirectory contains other directories (which would be the actual classes)
    # and is not explicitly named 'train' or 'test'
    if single_subdir_name not in ['train', 'test'] and len([d for d in os.listdir(directories_in_root[0]) if os.path.isdir(os.path.join(directories_in_root[0], d))]) > 0:
        base_search_path = directories_in_root[0]
        print(f"[INFO] Adjusted base data path to (single nested directory): {base_search_path}")

# Now, construct possible_data_roots relative to the (potentially adjusted) base_search_path
possible_data_roots = [os.path.join(base_search_path, 'train'), base_search_path]

for root_path in possible_data_roots:
    if os.path.isdir(root_path):
        for class_name in os.listdir(root_path):
            class_path = os.path.join(root_path, class_name)
            if os.path.isdir(class_path) and not class_name.startswith('.'):
                 # Check if the folder contains images
                if any(f.lower().endswith(('.jpg', '.jpeg', '.png')) for f in os.listdir(class_path)):
                    LABELS.add(class_name)

if not LABELS:
    raise ValueError("No class directories found in the dataset. Check extraction structure.")

LABELS = sorted(list(LABELS)) # Sort for consistent order
NUM_CLASSES = len(LABELS)
print(f"[INFO] Discovered {NUM_CLASSES} labels: {LABELS}")

data, labels_list = [], []
print("[INFO] Loading and preprocessing images from folders...")

# Iterate through class folders
image_count = 0
for i, label in enumerate(LABELS):
    for root_path in possible_data_roots:
        class_folder = os.path.join(root_path, label)
        if os.path.isdir(class_folder):
            for img_file in os.listdir(class_folder):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    imagePath = os.path.join(class_folder, img_file)
                    image = preprocess_with_otsu(imagePath)

                    if image is not None:
                        # Convert to RGB before appending
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        data.append(image)
                        labels_list.append(label)
                        image_count += 1
                        if image_count % 500 == 0:
                            print(f"  Loaded {image_count} images...")

if len(data) == 0:
    raise ValueError("No images were loaded. Check your EXTRACT_FOLDER name and the dataset structure.")

data = np.array(data, dtype="float32")
labels_list = np.array(labels_list)

# 🔠 Encode labels
lb = LabelBinarizer()
labels_encoded = lb.fit_transform(labels_list)

# Handle single class case for binarizer shape compatibility
if NUM_CLASSES == 1:
    labels_encoded = np.expand_dims(labels_encoded, axis=1)

# 🔀 Train-test split
(trainX, testX, trainY, testY) = train_test_split(
    data, labels_encoded, test_size=TEST_SPLIT, stratify=labels_encoded, random_state=42
)

print(f"\n[SUMMARY] Training samples: {len(trainX):,} | Test samples: {len(testX):,}")

# 🔄 Data augmentation setup (Mean subtraction for VGG19)
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug = ImageDataGenerator(rotation_range=30, zoom_range=0.15, width_shift_range=0.2,
                             height_shift_range=0.2, shear_range=0.15, horizontal_flip=True,
                             fill_mode="nearest")
valAug = ImageDataGenerator()
trainAug.mean = mean
valAug.mean = mean

# --- PART 2: BUILD, COMPILE, AND TRAIN MODEL ---
print("\n--- PART 2: Build, Compile, and Train Model ---")
print("[INFO] Building and compiling VGG19 model...")

# VGG19 Model definition (Transfer Learning)
baseModel = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.4)(headModel)
# Output layer: softmax for multi-class, sigmoid for binary
activation_fn = "softmax" if NUM_CLASSES > 1 else "sigmoid"
headModel = Dense(NUM_CLASSES, activation=activation_fn)(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

# Freeze base model layers
for layer in baseModel.layers:
    layer.trainable = False

# Compile and train
loss_fn = "categorical_crossentropy" if NUM_CLASSES > 1 else "binary_crossentropy"
opt = Adam(learning_rate=1e-3)
model.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy"])

print(f"[INFO] Training VGG19 model for {EPOCHS} epochs...")
H = model.fit(trainAug.flow(trainX, trainY, batch_size=BATCH_SIZE),
              steps_per_epoch=max(1, len(trainX) // BATCH_SIZE),
              validation_data=valAug.flow(testX, testY),
              validation_steps=max(1, len(testX) // BATCH_SIZE),
              epochs=EPOCHS)

# --- PART 3: EVALUATION AND SAVING ---
print("\n--- PART 3: Evaluation and Saving ---")

# Evaluation
print("[INFO] Evaluating model...")
predictions = model.predict(valAug.flow(testX, testY, batch_size=BATCH_SIZE), steps=len(testX) // BATCH_SIZE + 1)

# Determine the final predicted indices for classification report
if NUM_CLASSES > 1:
    y_true = testY.argmax(axis=1)
    y_pred = predictions.argmax(axis=1)
else: # Binary case
    y_true = testY.flatten()
    y_pred = (predictions > 0.5).astype(int).flatten()

print(classification_report(y_true, y_pred, target_names=lb.classes_))

# Save model and label binarizer
MODEL_FILE = "wheat_model.h5"
LB_FILE = "wheat_lb.pickle"

try:
    model.save(MODEL_FILE)
    with open(LB_FILE, "wb") as f:
        f.write(pickle.dumps(lb))
    print(f"[SUCCESS] Model and label binarizer saved as '{MODEL_FILE}' and '{LB_FILE}'.")
except Exception as e:
    print(f"[FATAL] Could not save model files: {e}. Check directory permissions.")

# Accuracy and Loss Plots
plt.style.use("ggplot")
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, EPOCHS), H.history['accuracy'], label="Train Acc")
plt.plot(np.arange(0, EPOCHS), H.history['val_accuracy'], label="Val Acc")
plt.title("VGG19 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.arange(0, EPOCHS), H.history['loss'], label="Train Loss")
plt.plot(np.arange(0, EPOCHS), H.history['val_loss'], label="Val Loss")
plt.title("VGG19 Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show() #

# --- PART 4: LIVE PREDICTION DEMONSTRATION ---
print("\n--- PART 4: Live Prediction Demonstration ---")

if not (os.path.exists(MODEL_FILE) and os.path.exists(LB_FILE)) or len(testX) == 0:
    print("[DEMO FAIL] Required files or test data not available. Skipping prediction demo.")
else:
    try:
        loaded_model = load_model(MODEL_FILE)
        with open(LB_FILE, "rb") as f:
            loaded_lb = pickle.loads(f.read())

        # Get a sample image (the first test image)
        sample_index = 0
        sample_image = testX[sample_index]

        # Determine true label
        true_label_idx = testY[sample_index].argmax() if NUM_CLASSES > 1 else int(testY[sample_index][0])
        true_label = loaded_lb.classes_[true_label_idx]

        # Prepare the image for prediction
        image_to_predict = np.expand_dims(sample_image, axis=0)
        image_to_predict = image_to_predict.astype("float32") - mean

        # Make the prediction
        prediction = loaded_model.predict(image_to_predict, verbose=0)[0]

        # Determine predicted label and confidence
        if NUM_CLASSES > 1:
            i = np.argmax(prediction)
            confidence = prediction[i] * 100
        else:
            i = int(prediction[0] > 0.5)
            confidence = prediction[0] if i == 1 else (1.0 - prediction[0])
            confidence *= 100

        predicted_label = loaded_lb.classes_[i]

        # Display results
        print("------------------------------------------")
        print(f"Sample Image True Label: {true_label}")
        print(f"Predicted Label: {predicted_label}")
        print(f"Confidence: {confidence:.2f}%")
        print("------------------------------------------")

        plt.imshow(sample_image.astype(np.uint8))
        plt.title(f"Prediction: {predicted_label} ({confidence:.1f}%)")
        plt.axis("off")
        plt.show() #

    except Exception as e:
        print(f"[DEMO FAIL] An error occurred during the prediction demo: {e}")

[SETUP] Downloading dataset lyxbash/lcdcd-2020-dataset using KaggleHub...
Using Colab cache for faster access to the 'lcdcd-2020-dataset' dataset.
[SUCCESS] Dataset downloaded and extracted to: /kaggle/input/lcdcd-2020-dataset

--- PART 1: Data Loading and Preparation ---
[INFO] Adjusted base data path to (single nested directory): /kaggle/input/lcdcd-2020-dataset/Large Wheat Disease Classification Dataset
[INFO] Discovered 4 labels: ['Crown and Root Rot', 'Healthy Wheat', 'Leaf Rust', 'Wheat Loose Smut']
[INFO] Loading and preprocessing images from folders...
  Loaded 500 images...
  Loaded 1000 images...
  Loaded 1500 images...
  Loaded 2000 images...
  Loaded 2500 images...
  Loaded 3000 images...
  Loaded 3500 images...

[SUMMARY] Training samples: 2,670 | Test samples: 891

--- PART 2: Build, Compile, and Train Model ---
[INFO] Building and compiling VGG19 model...
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[INFO] Training VGG19 model for 50 epochs...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
12/41 ━━━━━━━━━━━━━━━━━━━━ 14:32 30s/step - accuracy: 0.3289 - loss: 4.1202

In [ ]:
import os
import shutil

# Define the target dataset path
dataset_path = '/content/Dataset'

# Create the Dataset directory if it doesn't exist
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
    print(f"Created directory: {dataset_path}")

# Move 'train' and 'test' directories into 'Dataset'
source_train = '/content/train'
source_test = '/content/test'

if os.path.exists(source_train) and os.path.isdir(source_train):
    shutil.move(source_train, dataset_path)
    print(f"Moved {source_train} to {dataset_path}")
else:
    print(f"Warning: {source_train} not found. Skipping move.")

if os.path.exists(source_test) and os.path.isdir(source_test):
    shutil.move(source_test, dataset_path)
    print(f"Moved {source_test} to {dataset_path}")
else:
    print(f"Warning: {source_test} not found. Skipping move.")

# Verify the new structure
print("Current structure of /content/Dataset:")
!ls -F {dataset_path}

Created directory: /content/Dataset
Current structure of /content/Dataset:


In [ ]:
!ls -F /content/

sample_data/  test/  test.csv  train/  train.csv


In [ ]:
# 📦 Install dependencies
!pip install opencv-python tensorflow imutils scikit-learn matplotlib pandas

# 📁 Import libraries
import cv2, os, pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipfile

# --- FIXED TENSORFLOW IMPORTS ---
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
# ---------------------------------

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# --- CONFIGURATION (CRITICAL) ---
# Assuming your zip file name is correct:
ZIP_FILE_NAME = "mango-leaf-disease-dataset.zip"
EXTRACT_FOLDER = "mango-leaf-data"
KAGGLE_PREFIX = "/kaggle/input/"
# ---------------------------------

# --- UNZIP THE DATASET ---
print(f"[SETUP] Starting extraction of {ZIP_FILE_NAME}...")
try:
    if os.path.exists(ZIP_FILE_NAME):
        with zipfile.ZipFile(ZIP_FILE_NAME, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_FOLDER)
        print(f"[SUCCESS] Dataset extracted to the folder: {EXTRACT_FOLDER}/")
    else:
        print(f"[WARNING] Zip file '{ZIP_FILE_NAME}' not found. Assuming data is already extracted.")
except Exception as e:
    print(f"[FATAL] Error during unzipping: {e}")
    raise SystemExit

# --- 🧠 Otsu Segmentation Function (Preprocessing) ---
def preprocess_with_otsu(image_path):
    """Loads, segments using Otsu's thresholding, and resizes an image."""
    image = cv2.imread(image_path)
    if image is None:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return cv2.resize(segmented, (224, 224))

# --- PART 1: DATA LOADING AND PREPARATION ---
print("\n--- PART 1: Data Loading and Preparation ---")

# Dynamically determine LABELS from the dataset structure
LABELS = set()
train_data_path = os.path.join(EXTRACT_FOLDER, 'train')
test_data_path = os.path.join(EXTRACT_FOLDER, 'test') # Assuming there might be a test set too

if os.path.isdir(train_data_path):
    for class_name in os.listdir(train_data_path):
        if os.path.isdir(os.path.join(train_data_path, class_name)):
            LABELS.add(class_name)

if os.path.isdir(test_data_path):
    for class_name in os.listdir(test_data_path):
        if os.path.isdir(os.path.join(test_data_path, class_name)):
            LABELS.add(class_name)

if not LABELS:
    raise ValueError("No class directories found in the dataset. Please check your extracted data structure.")

LABELS = sorted(list(LABELS)) # Sort for consistent order
print(f"[INFO] Discovered labels: {LABELS}")


# Load the CSV
try:
    df = pd.read_csv(os.path.join(EXTRACT_FOLDER, "train.csv"))
except FileNotFoundError:
    raise FileNotFoundError("train.csv not found. Ensure it is uploaded and correctly extracted.")

data, labels = [], []
print("[INFO] Loading and preprocessing images from CSV...")

# Iterate over the DataFrame to load and preprocess images
for index, row in df.iterrows():
    original_path = row['filename'] # e.g., /kaggle/input/mango-leaf-disease-dataset/Powdery Mildew/IMG_...jpg

    # Locate the part of the path that comes after the dataset root in the original Kaggle structure
    path_components = original_path.split(os.path.sep)
    try:
        # Find 'mango-leaf-disease-dataset' and take components after it
        dataset_name_index = path_components.index('mango-leaf-disease-dataset')
        # The relevant part is 'Powdery Mildew/IMG_...jpg'
        relevant_path_tail = os.path.join(*path_components[dataset_name_index + 1:])
    except ValueError:
        # If 'mango-leaf-disease-dataset' not found, assume it's already a relative path to class folder
        # This case is less likely if original_path is always from Kaggle
        relevant_path_tail = original_path

    # Construct the final path: EXTRACT_FOLDER / 'train' / CLASS_NAME / IMAGE_NAME
    # Assuming all images in train.csv belong to the 'train' split
    imagePath = os.path.join(EXTRACT_FOLDER, 'train', relevant_path_tail)

    label = row['label']

    if label not in LABELS:
        print(f"[WARNING] Skipping image {imagePath} with label '{label}' not in discovered LABELS.")
        continue

    image = preprocess_with_otsu(imagePath)

    if image is None:
        # File not found or couldn't be read - usually due to path error
        # print(f"[ERROR] Image not found or could not be loaded: {imagePath}. Skipping.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    data.append(image)
    labels.append(label)

if len(data) == 0:
    raise ValueError("No images were loaded. Check your ZIP_FILE_NAME and EXTRACT_FOLDER name, and image paths in train.csv.")

data = np.array(data, dtype="float32")
labels = np.array(labels)

# 🔠 Encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# 🔀 Train-test split
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

# 🔄 Data augmentation setup (Mean subtraction for VGG19)
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug = ImageDataGenerator(rotation_range=30, zoom_range=0.15, width_shift_range=0.2,
                              height_shift_range=0.2, shear_range=0.15, horizontal_flip=True,
                              fill_mode="nearest")
valAug = ImageDataGenerator()
trainAug.mean = mean
valAug.mean = mean

# --- PART 2: BUILD, COMPILE, AND TRAIN MODEL ---
print("\n--- PART 2: Build, Compile, and Train Model ---")
print("[INFO] Building and compiling VGG19 model...")

# VGG19 Model definition
baseModel = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.4)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

# Compile and train
opt = Adam(learning_rate=1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
BATCH_SIZE = 64
EPOCHS = 50

print(f"[INFO] Training VGG19 model for {EPOCHS} epochs on {len(data)} images...")
H = model.fit(trainAug.flow(trainX, trainY, batch_size=BATCH_SIZE),
              steps_per_epoch=len(trainX) // BATCH_SIZE,
              validation_data=valAug.flow(testX, testY),
              validation_steps=len(testX) // BATCH_SIZE,
              epochs=EPOCHS)

# --- PART 3: EVALUATION AND SAVING ---
print("\n--- PART 3: Evaluation and Saving ---")

# Evaluation
print("[INFO] Evaluating model...")
predictions = model.predict(valAug.flow(testX, testY, batch_size=BATCH_SIZE), steps=len(testX) // BATCH_SIZE + 1)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))
#

# Save model and label binarizer
try:
    model.save("model.h5")
    with open("lb.pickle", "wb") as f:
        f.write(pickle.dumps(lb))
    print("[SUCCESS] Model and label binarizer saved as 'model.h5' and 'lb.pickle'.")
except Exception as e:
    print(f"[FATAL] Could not save model files: {e}. Prediction demo will likely fail.")

# Accuracy and Loss Plots
plt.style.use("ggplot")
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, EPOCHS), H.history['accuracy'], label="Train Acc")
plt.plot(np.arange(0, EPOCHS), H.history['val_accuracy'], label="Val Acc")
plt.title("VGG19 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.arange(0, EPOCHS), H.history['loss'], label="Train Loss")
plt.plot(np.arange(0, EPOCHS), H.history['val_loss'], label="Val Loss")
plt.title("VGG19 Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()
#

# --- PART 4: LIVE PREDICTION DEMONSTRATION (WITH ERROR CHECKING) ---
print("\n--- PART 4: Live Prediction Demonstration ---")

MODEL_FILE = "model.h5"
LB_FILE = "lb.pickle"

if not (os.path.exists(MODEL_FILE) and os.path.exists(LB_FILE)):
    print(f"[DEMO FAIL] Required files '{MODEL_FILE}' or '{LB_FILE}' were not found. Skipping prediction demo.")
elif len(testX) == 0:
    print("[WARNING] Not enough test images to run a prediction demonstration.")
else:
    try:
        # Reload model and lb
        print("[INFO] Reloading saved model and label binarizer for demo...")
        loaded_model = load_model(MODEL_FILE)
        with open(LB_FILE, "rb") as f:
            loaded_lb = pickle.loads(f.read())

        # Get a sample image and its ground truth label
        sample_image = testX[0]
        true_label = loaded_lb.classes_[testY[0].argmax()]

        # Prepare the image for prediction
        image_to_predict = np.expand_dims(sample_image, axis=0)
        image_to_predict = image_to_predict.astype("float32") - mean

        # Make the prediction
        print("[INFO] Making prediction on a sample test image...")
        prediction = loaded_model.predict(image_to_predict)[0]
        i = np.argmax(prediction)
        predicted_label = loaded_lb.classes_[i]
        confidence = prediction[i] * 100

        # Display results
        print("------------------------------------------")
        print(f"Sample Image True Label: {true_label}")
        print(f"Predicted Label: {predicted_label}")
        print(f"Confidence: {confidence:.2f}%")
        print("------------------------------------------")

        plt.imshow(sample_image.astype(np.uint8))
        plt.title(f"Prediction: {predicted_label} ({confidence:.1f}%)")
        plt.axis("off")
        plt.show()

    except Exception as e:
        print(f"[DEMO FAIL] An error occurred during the prediction demo: {e}")

[SETUP] Starting extraction of mango-leaf-disease-dataset.zip...
[SUCCESS] Dataset extracted to the folder: mango-leaf-data/

--- PART 1: Data Loading and Preparation ---
[INFO] Discovered labels: ['Anthracnose', 'Bacterial Canker', 'Cutting Weevil', 'Die Back', 'Gall Midge', 'Healthy', 'Powdery Mildew', 'Sooty Mould']
[INFO] Loading and preprocessing images from CSV...

--- PART 2: Build, Compile, and Train Model ---
[INFO] Building and compiling VGG19 model...
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
[INFO] Training VGG19 model for 50 epochs on 3200 images...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1813s 49s/step - accuracy: 0.4185 - loss: 2.5793 - val_accuracy: 0.7370 - val_loss: 0.7418
Epoch 2/50
 1/37 ━━━━━━━━━━━━━━━━━━━━ 24:12 40s/step - accuracy: 0.7188 - loss: 0.8964

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


37/37 ━━━━━━━━━━━━━━━━━━━━ 281s 7s/step - accuracy: 0.7188 - loss: 0.8964 - val_accuracy: 0.7370 - val_loss: 0.7404
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1762s 48s/step - accuracy: 0.6711 - loss: 0.9955 - val_accuracy: 0.7786 - val_loss: 0.6424
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 280s 7s/step - accuracy: 0.7812 - loss: 0.7731 - val_accuracy: 0.7552 - val_loss: 0.6615
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1764s 48s/step - accuracy: 0.7052 - loss: 0.7821 - val_accuracy: 0.8229 - val_loss: 0.4975
Epoch 6/50
 1/37 ━━━━━━━━━━━━━━━━━━━━ 24:10 40s/step - accuracy: 0.7656 - loss: 0.6791

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


37/37 ━━━━━━━━━━━━━━━━━━━━ 328s 8s/step - accuracy: 0.7656 - loss: 0.6791 - val_accuracy: 0.8438 - val_loss: 0.4687
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1812s 49s/step - accuracy: 0.7492 - loss: 0.7021 - val_accuracy: 0.8125 - val_loss: 0.5363
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 283s 7s/step - accuracy: 0.7500 - loss: 0.5642 - val_accuracy: 0.8047 - val_loss: 0.5509
Epoch 9/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1766s 48s/step - accuracy: 0.7734 - loss: 0.6332 - val_accuracy: 0.8411 - val_loss: 0.4956
Epoch 10/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 321s 8s/step - accuracy: 0.7969 - loss: 0.5778 - val_accuracy: 0.8490 - val_loss: 0.4851
Epoch 11/50
33/37 ━━━━━━━━━━━━━━━━━━━━ 2:43 41s/step - accuracy: 0.8003 - loss: 0.5376

KeyboardInterrupt: 

In [ ]:
!unzip -l {ZIP_FILE_NAME}

Archive:  mango-leaf-disease-dataset.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
    78081  2023-04-27 04:10   test.csv
    33930  2023-04-27 04:10   test/Anthracnose/20211008_124305 (Custom).jpg
    32433  2023-04-27 04:10   test/Anthracnose/20211008_124306 (Custom).jpg
    32907  2023-04-27 04:10   test/Anthracnose/20211008_124341 (Custom).jpg
    34064  2023-04-27 04:10   test/Anthracnose/20211008_124501 (Custom).jpg
    33760  2023-04-27 04:10   test/Anthracnose/20211008_124931 (Custom).jpg
    36844  2023-04-27 04:10   test/Anthracnose/20211008_124933 (Custom).jpg
    37973  2023-04-27 04:10   test/Anthracnose/20211008_125014 (Custom).jpg
    34740  2023-04-27 04:10   test/Anthracnose/20211008_125052 (Custom).jpg
    38574  2023-04-27 04:10   test/Anthracnose/20211008_125112 (Custom).jpg
    40914  2023-04-27 04:10   test/Anthracnose/20211011_131259 (Custom).jpg
    31127  2023-04-27 04:10   test/Anthracnose/20211011_133431 (Custom).jpg
    30744  202